# Chain-of-Thought (CoT) Reasoning

## Overview

**Chain-of-Thought prompting** improves LLM accuracy on complex tasks by eliciting step-by-step reasoning. This technique is critical for:

1. **Multi-Step Problems** - Math, logic, troubleshooting
2. **Fact-Checking** - Verify assumptions against data
3. **Policy Interpretation** - Complex rule-based decisions
4. **Debugging** - Understand LLM's reasoning process

## Why CoT Matters

### Accuracy Improvements
- **Math Problems**: 50-80% accuracy gain
- **Logic Puzzles**: 40-60% improvement  
- **Multi-Step Reasoning**: 30-50% better results

### Transparency
- See **how** the LLM reached its conclusion
- Debug incorrect outputs
- Build user trust with explainability

## Two CoT Patterns

### 1. Visible Reasoning
Show step-by-step thinking to the user (educational, transparency)

### 2. Inner Monologue (This Notebook)
Hide reasoning steps, show only final answer (better UX, cleaner responses)

## Key Concepts

### Structured Prompts
Break tasks into numbered steps with clear instructions

### Delimiter Separation
Use markers (e.g., `####`) to separate reasoning from answer

### Inner Monologue
LLM reasons internally, user sees only final output

## Environment Setup

In [ ]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

---

## Structured CoT System Message

This system message implements a **5-step reasoning process** for customer service queries:

### Step-by-Step Breakdown

1. **Classify Intent** - Is user asking about specific products?
2. **Validate Products** - Are mentioned products in our catalog?
3. **Extract Assumptions** - What is the user assuming? (e.g., price comparison)
4. **Fact-Check** - Are assumptions correct based on product data?
5. **Respond** - Correct misconceptions, answer question

### Key Features

- **Product Catalog** - Embedded in system message (5 laptops)
- **Delimiter Format** - Forces structured output: `Step 1:#### ... Step 2:#### ...`
- **Error Correction** - Identifies and fixes user misconceptions
- **Tone Control** - "Polite and friendly" specified

**Production Note**: In real systems, product data comes from database/API, not embedded in prompt.

In [ ]:
client = openai.OpenAI()

def get_completion_from_messages(
    messages,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content


## Test Case 1: Price Comparison with False Assumption

User assumes Chromebook ($249.99) is MORE expensive than Desktop ($999.99) - **incorrect**.

CoT will:
1. Identify the two products
2. Detect the false assumption
3. Correct it politely
4. Provide accurate price comparison

In [ ]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

**Expected Response** (with reasoning steps visible):
```
Step 1:#### User is asking about specific products
Step 2:#### Both products exist in catalog
Step 3:#### User assumes Chromebook is more expensive  
Step 4:#### Assumption is FALSE (Chromebook $249.99 < Desktop $999.99)
Response to user:#### Actually, the BlueWave Chromebook is $750 LESS expensive than the TechPro Desktop...
```

**Key Benefit**: CoT catches the reversed assumption and corrects it.

---

## Test Case 2: Out-of-Scope Query

In [ ]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages, "gpt-3.5-turbo")
print(response)

**Expected Response**:
```
Step 1:#### Not asking about specific products
Step 2:#### N/A
Step 3:#### N/A
Step 4:#### N/A
Response to user:#### No, we don't sell TVs. We specialize in computers and laptops.
```

**Key Benefit**: CoT determines question is off-topic, responds accordingly.

---

## Inner Monologue: Hiding Reasoning Steps

The full CoT reasoning (Steps 1-4) is useful for debugging but clutters the user experience.

**Solution**: Extract only the final response using delimiter splitting.

In [ ]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Inner Monologue
Since we asked the LLM to separate its reasoning steps by a delimiter, we can hide the chain-of-thought reasoning from the final output that the user sees.

In [ ]:
# Extract only the final response (after last delimiter)
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    # Fallback if parsing fails
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)  # User sees ONLY this, reasoning hidden

**Result**: Clean, user-friendly response without intermediate reasoning steps.

---

## Production Implementation Patterns

### Complete CoT Pipeline

```python
def process_with_cot(user_query, product_catalog):
    # 1. Build CoT system message
    system_message = f\"\"\"
    Follow these steps:
    Step 1: Identify intent
    Step 2: Validate against catalog
    Step 3: Extract assumptions
    Step 4: Fact-check assumptions
    Response to user: Final answer
    
    Use delimiter {delimiter} between steps.
    
    Product Catalog:
    {product_catalog}
    \"\"\"
    
    # 2. Get full reasoning
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_query}{delimiter}"}
    ]
    full_response = get_completion_from_messages(messages)
    
    # 3. Log full reasoning (for debugging/auditing)
    log_cot_reasoning(user_query, full_response)
    
    # 4. Extract final answer for user
    final_answer = extract_final_response(full_response, delimiter)
    
    return final_answer
```

### When to Use CoT

| Task Type | Use CoT? | Reasoning |
|-----------|----------|-----------|
| Simple FAQ | ❌ No | Overhead not justified |
| Math calculations | ✅ Yes | Accuracy critical |
| Multi-step troubleshooting | ✅ Yes | Needs structured thinking |
| Policy interpretation | ✅ Yes | Must verify compliance |
| Creative writing | ❌ No | Constrains creativity |
| Product recommendations | ✅ Yes | Compare multiple criteria |

### Cost Considerations

**Token Overhead**:
- CoT adds ~100-200 tokens per request (reasoning steps)
- Cost: ~$0.0001-0.0002 per request (gpt-3.5-turbo)

**Worth It When**:
- Accuracy gain > 20%
- User questions are complex
- Debugging/auditing is important

**Not Worth It When**:
- Simple lookups
- High-volume, low-complexity queries
- Cost sensitivity outweighs accuracy

### Debugging with CoT

```python
# Development: Show full reasoning
if DEBUG_MODE:
    print("=== CoT Reasoning ===")
    print(full_response)
    print("=== Final Response ===")
    print(final_answer)

# Production: Log reasoning to analytics
analytics.log_cot({
    "user_query": user_query,
    "reasoning": full_response,
    "final_answer": final_answer,
    "timestamp": datetime.now()
})
```

---

## Advanced CoT Techniques

### 1. Few-Shot CoT
Provide examples of correct reasoning:

```python
system_message = \"\"\"
Example 1:
User: Is Product A cheaper than Product B?
Step 1: Both products exist
Step 2: A=$100, B=$200
Step 3: User asks comparison
Step 4: A < B, so yes
Response: Yes, Product A ($100) is $100 cheaper.

Now answer this query:
{user_query}
\"\"\"
```

### 2. Self-Consistency
Generate multiple CoT reasoning paths, take majority vote:

```python
responses = [
    get_completion_with_cot(query, temperature=0.7) 
    for _ in range(5)
]
final_answer = majority_vote(responses)
```

### 3. Least-to-Most Prompting
Break complex problem into subproblems:

```python
# Step 1: List subproblems
subproblems = get_completion("Break this into steps: {query}")

# Step 2: Solve each
solutions = [solve(sub) for sub in subproblems]

# Step 3: Combine
final = combine_solutions(solutions)
```

---

## Summary: CoT Best Practices

### Implementation
1. **Structure Steps Clearly** - Number them, define each purpose
2. **Use Delimiters** - Separate reasoning from answer
3. **Hide Reasoning** - Extract final response for UX
4. **Log Full Output** - Keep for debugging/auditing
5. **Error Handling** - Fallback if parsing fails

### When to Use
- ✅ Multi-step reasoning
- ✅ Fact-checking required
- ✅ Policy/rule interpretation
- ✅ Debugging needed
- ❌ Simple lookups
- ❌ Creative/open-ended tasks

### Performance
- **Accuracy**: +30-80% on complex tasks
- **Cost**: +50-100 tokens overhead
- **Latency**: +0.5-1s (more thinking)

### Next Steps
- **E.PromptChaining** - Combine CoT with multi-step workflows
- **F.EvaluatingOutputs** - Assess CoT reasoning quality
- **G.CustomerServiceBot** - Integrate CoT into full system